In [1]:
import numpy as np  # sometimes needed to avoid mkl-service error
import sys
import os
import argparse
import logging
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import CSVLogger, WandbLogger
from pytorch_lightning.plugins import DDPPlugin
from pytorch_lightning.utilities import rank_zero_only
import torch
from torchmdnet.module import LNNP
from torchmdnet import datasets, priors, models
from torchmdnet.data import DataModule
from torchmdnet.models import output_modules
from torchmdnet.models.utils import rbf_class_mapping, act_class_mapping
from torchmdnet.utils import LoadFromFile, LoadFromCheckpoint, save_argparse, number
from pathlib import Path
import wandb
import json
import pandas as pd
from rdkit.Chem import AllChem
import copy
from rdkit.Geometry import Point3D
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import py3Dmol
from rdkit.Chem import rdDepictor
from rdkit.Chem import rdDistGeom
import rdkit

In [31]:
command_dic={
"command_args_baselinedefault_defnoise":'command_args_baselinedefault_defnoise.txt',
"command_args_baselinedefault_rdkitnoise":"command_args_baselinedefault_rdkitnoise.txt",
"command_args_baselinedefault_rdkit04noise":"command_args_baselinedefault_rdkit04noise.txt",
"command_args_baselinedefault_8noise":"command_args_baselinedefault_8noise.txt",
"command_args_baseline8noise_defnoise":"command_args_baseline8noise_defnoise.txt",
"command_args_baseline8noise_rdkitnoise":"command_args_baseline8noise_rdkitnoise.txt",
"command_args_baseline8noise_rdkit04noise":"command_args_baseline8noise_rdkit04noise.txt",
"command_args_baseline8noise_8noise":"command_args_baseline8noise_8noise.txt",


"command_args_oursdefault_defnoise":"command_args_oursdefault_defnoise.txt",
"command_args_oursdefault_rdkitnoise":"command_args_oursdefault_rdkitnoise.txt",
"command_args_oursdefault_rdkit04noise":"command_args_oursdefault_rdkit04noise.txt",
"command_args_oursdefault_8noise":"command_args_oursdefault_8noise.txt",
"command_args_ours04noise_defnoise":"command_args_ours04noise_defnoise.txt",
"command_args_ours04noise_8noise":"command_args_ours04noise_8noise.txt",
"command_args_ours04noise_rdkitnoise":"command_args_ours04noise_rdkitnoise.txt",
"command_args_ours04noise_rdkit04noise":"command_args_ours04noise_rdkit04noise.txt",}




In [38]:
def load_data(argfile):
    root="arg_files/"
    with open(root+argfile, 'r') as f:
        args = json.load(f)
    data = DataModule(args)
    data.prepare_data()
    data.setup("fit")
    return data,args

In [40]:
data_defnoise,arg_baseline=load_data(command_dic["command_args_baselinedefault_defnoise"])
data_8noise,_=load_data(command_dic["command_args_baselinedefault_8noise"])


c:\Users\aminr\.conda\envs\pvd\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


train 400, val 50, test 37


computing mean and std:   0%|          | 0/4 [00:00<?, ?it/s]c:\Users\aminr\Documents\Thesis\pre-training-via-denoising\torchmdnet\data.py:172: UserWarning: Standardize is true but failed to compute dataset mean and standard deviation. Maybe the dataset only contains forces.
  rank_zero_warn(
computing mean and std:   0%|          | 0/4 [00:00<?, ?it/s]


train 400, val 50, test 37


computing mean and std:   0%|          | 0/4 [00:00<?, ?it/s]


In [10]:
data.dataset_maybe_noisy[0].pos

tensor([[-0.0176,  1.0177, -0.0019],
        [ 0.0280, -0.0533,  0.0182],
        [ 0.9882,  1.4933,  0.0269],
        [-0.5864,  1.4333, -0.9694],
        [-0.4801,  1.4544,  0.8777]])

In [11]:
sample=data.dataset_maybe_noisy[0]

In [12]:
sample.pos- sample.pos_target

tensor([[-1.2700e-02,  1.0858e+00,  8.0000e-03],
        [ 2.2000e-03, -6.0000e-03,  2.0000e-03],
        [ 1.0117e+00,  1.4638e+00,  3.0000e-04],
        [-5.4080e-01,  1.4475e+00, -8.7660e-01],
        [-5.2380e-01,  1.4379e+00,  9.0640e-01]])

In [13]:
model = LNNP(args, prior_model=None, mean=data.mean, std=data.std)
model.eval()

LNNP(
  (model): TorchMD_Net(
    (representation_model): TorchMD_ET(hidden_channels=256, num_layers=8, num_rbf=64, rbf_type=expnorm, trainable_rbf=False, activation=silu, attn_activation=silu, neighbor_embedding=NeighborEmbeddingJittable_f8da98(
      (embedding): Embedding(100, 256)
      (distance_proj): Linear(in_features=64, out_features=256, bias=True)
      (combine): Linear(in_features=512, out_features=256, bias=True)
      (cutoff): CosineCutoff()
    ), num_heads=8, distance_influence=both, cutoff_lower=0.0, cutoff_upper=5.0)
    (output_model): EquivariantScalar(
      (output_network): ModuleList(
        (0): GatedEquivariantBlock(
          (vec1_proj): Linear(in_features=256, out_features=256, bias=False)
          (vec2_proj): Linear(in_features=256, out_features=128, bias=False)
          (update_net): Sequential(
            (0): Linear(in_features=512, out_features=256, bias=True)
            (1): SiLU()
            (2): Linear(in_features=256, out_features=256, bia

In [15]:
_,noise,_=model(sample.z,sample.pos)

In [16]:
def make_conformer(mol,positions):
    id=mol.GetNumConformers()
    conformer = Chem.Conformer(mol.GetNumAtoms())
    conformer.SetId(id)
    for i in range(mol.GetNumAtoms()):
        conformer.SetAtomPosition(i, Point3D(positions[i][0].item(),positions[i][1].item(),positions[i][2].item())) 
    mol.AddConformer(conformer)

In [20]:
sample.mol.RemoveAllConformers()
predpos=sample.pos-noise
gpos=sample.pos-sample.pos_target
initpos=sample.pos
make_conformer(sample.mol,predpos)
make_conformer(sample.mol,gpos)
make_conformer(sample.mol,initpos)

In [23]:
sample.mol.GetNumConformers()

3

In [25]:
w = Chem.SDWriter("result/"+sample.name+".sdf")
for id in range(sample.mol.GetNumConformers()):
    w.write(sample.mol,confId=id)
w.close()

In [28]:
IPythonConsole.drawMol3D(sample.mol,confId=2)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol